# Reports

In [13]:
import os
import re
from datetime import datetime
current_datetime = datetime.now().strftime('%Y%m%d.%H%M')

import numpy as np
import pandas as pd

# for the notebook rendering 
from IPython.display import display, HTML, FileLink, Markdown
from IPython.display import FileLink as FL
from IPython.display import Markdown as MD


DIR = os.getcwd()
print(f'{DIR=}')

RDIR = os.path.join(DIR,'reports')
print(f'{RDIR=}')

DIR='c:\\Users\\JGarza\\GitHub\\wifi_monitor'
RDIR='c:\\Users\\JGarza\\GitHub\\wifi_monitor\\reports'


In [14]:
def break_down_buckets(idf,column,buckets,message='',nan_value=-1):
    """
    breaks a column down into buckets/bins
    """
    idf = idf.fillna(nan_value)

    print('',message,'\ncolumn: ',column, '\nbuckets: ', buckets)
    
    idf = pd.DataFrame(idf[column])
    idf['bucket'] = pd.cut(idf[column], bins=buckets)
    idf = idf.groupby(by='bucket').count()
    idf['percent'] = (idf[column]/idf[column].sum())*100
    idf['percent'] = idf['percent'].round(2)
    display(idf)

    return idf

#test
# break_down_buckets(df,'AGE_YRS',[-1,0,15,25,35,45,55,65,75,85,500])

In [15]:
def bar(num,denom=100.0,length=30,fillchar='#',emptychar='_'):
    fillnum = ((int)( (num/denom) * length))
    return '[' + ( fillnum * fillchar ).ljust(length,emptychar)  + ']' # + f" {(num/denom)*100.0:.2f}%     " 

In [16]:
# YYYYMMDDHHmm
DateFilter = None
DateFilter = 202501100000 # new firmware r44715
# DateFilter = 202501160000 # changed some settings in Error 404 NH

In [17]:
def getfirmware(ssid,dt):
    if 'NH' in ssid:
        
        if dt <= 202501100810:
            return "r58881"
        elif dt > 202501100811 and dt <= 202501102101:
            return "r44715"
        elif dt > 202501102101 : #and dt <= 203000000000:
            return "r58881"
    else:
        return ""

In [18]:
data = {}

for i in os.listdir(RDIR):
    # print(i)
    try:

        text = ''
        with open(os.path.join(RDIR, i), 'r') as f:
            text = f.read()
        # print(text)

        dt = re.findall('Datetime_alt: .*\n',text)[0]
        dt = dt.replace('Datetime_alt: ','').replace('\\n','')
        dt = int(dt)
            
        if DateFilter != None:
            if dt < DateFilter:
                #move on to next
                print('exclude',dt,DateFilter)
                continue
            else:
                print('include',dt,DateFilter)

        wifi = re.findall('^SSID:.*\n',text)[0]
        wifi = wifi.replace('\n', '').replace('SSID: ','')
        # wifi = wifi + " " + getfirmware(wifi,dt)

        pings = re.findall('.*time=.*\n',text)
        pings = [ re.sub('.*time=','',p) for p in pings]
        pings = [ re.sub('ms.*\n','',p) for p in pings]
        pings = [ int(p) for p in pings]
        # print(*pings,sep='\n')

        timeouts = re.findall('Request timed out.\n',text)

        trycount = re.findall('(Request|Reply).*\n',text)

        if wifi in data.keys():
            pass
        else:
            data[wifi] = {}
            data[wifi]['pings'] = []
            data[wifi]['timeouts'] = []
            data[wifi]['trycount'] = []
        
        data[wifi]['pings'].extend(pings)
        data[wifi]['timeouts'].extend([len(timeouts)])
        data[wifi]['trycount'].extend([len(trycount)])

    except Exception as e:
        pass
        print(i,e)

# for k in data.keys():
#     print('-'*20)
#     print(k)
#     df_pings = pd.DataFrame(data[k]['pings'])
#     print(df_pings[0].describe())
#     print()
#     print(f'Request timed out:')
#     print(f'{ sum(data[k]['timeouts']) } out of { sum(data[k]['trycount']) }')
#     print(f'{ sum(data[k]['timeouts']) / sum(data[k]['trycount']) }')

file = os.path.join(DIR,'docs','index.html')
with open(file,'w',encoding='utf8') as f:

    f.write(f'<h>{current_datetime}</h>')

    for k in data.keys():
        
        f.write('<p>' + '-'*20 + '</p>')

        f.write(f'<p>{k}</p>')
        df_pings = pd.DataFrame(data[k]['pings'])

        csvlines = df_pings.describe().to_csv()
        f.write(csvlines[4::].replace('\n','<br \>').replace(',','\t'))

        # print(df_pings)

        bdb = break_down_buckets(df_pings,0,[0,5,10,15,20,30,40,50,100,250,500])
        bdb['bar'] = bdb.percent.apply(bar)
        # display(bdb)

        f.write('<br \>')
        f.write(bdb.to_html())
        f.write('<br \>')

        f.write(f'Request timed out (failed pings):<br \>')
        f.write(f'{ sum(data[k]['timeouts']) } out of { sum(data[k]['trycount']) }<br \>')
        f.write(f'{ sum(data[k]['timeouts']) / sum(data[k]['trycount']) }')




Error404NH_202412292039.txt list index out of range
exclude 202412300625 202501100000
exclude 202412310635 202501100000
exclude 202412311659 202501100000
exclude 202501011257 202501100000
exclude 202501020539 202501100000
exclude 202501030416 202501100000
exclude 202501041159 202501100000
exclude 202501051039 202501100000
exclude 202501060604 202501100000
exclude 202501070520 202501100000
exclude 202501080600 202501100000
exclude 202501090551 202501100000
include 202501100810 202501100000
include 202501110123 202501100000
include 202501121229 202501100000
include 202501130644 202501100000
include 202501140522 202501100000
include 202501150633 202501100000
include 202501152246 202501100000
include 202501152256 202501100000
include 202501160552 202501100000
Error404_202412292041.txt list index out of range
exclude 202412300627 202501100000
exclude 202412310630 202501100000
exclude 202412311656 202501100000
exclude 202501011259 202501100000
exclude 202501020541 202501100000
exclude 202501

<>:77: SyntaxWarning: invalid escape sequence '\>'
<>:85: SyntaxWarning: invalid escape sequence '\>'
<>:87: SyntaxWarning: invalid escape sequence '\>'
<>:89: SyntaxWarning: invalid escape sequence '\>'
<>:90: SyntaxWarning: invalid escape sequence '\>'
<>:77: SyntaxWarning: invalid escape sequence '\>'
<>:85: SyntaxWarning: invalid escape sequence '\>'
<>:87: SyntaxWarning: invalid escape sequence '\>'
<>:89: SyntaxWarning: invalid escape sequence '\>'
<>:90: SyntaxWarning: invalid escape sequence '\>'
C:\Users\JGarza\AppData\Local\Temp\ipykernel_30736\3279194875.py:77: SyntaxWarning: invalid escape sequence '\>'
  f.write(csvlines[4::].replace('\n','<br \>').replace(',','\t'))
C:\Users\JGarza\AppData\Local\Temp\ipykernel_30736\3279194875.py:85: SyntaxWarning: invalid escape sequence '\>'
  f.write('<br \>')
C:\Users\JGarza\AppData\Local\Temp\ipykernel_30736\3279194875.py:87: SyntaxWarning: invalid escape sequence '\>'
  f.write('<br \>')
C:\Users\JGarza\AppData\Local\Temp\ipykernel_

,0,percent
bucket,,
"(0, 5]",0,0.00
"(5, 10]",176,19.56
"(10, 15]",590,65.56
"(15, 20]",67,7.44
"(20, 30]",29,3.22
"(30, 40]",11,1.22
"(40, 50]",4,0.44
"(50, 100]",5,0.56
"(100, 250]",12,1.33


  
column:  0 
buckets:  [0, 5, 10, 15, 20, 30, 40, 50, 100, 250, 500]


C:\Users\JGarza\AppData\Local\Temp\ipykernel_30736\3656564501.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  idf = idf.groupby(by='bucket').count()


,0,percent
bucket,,
"(0, 5]",0,0.00
"(5, 10]",372,53.14
"(10, 15]",272,38.86
"(15, 20]",27,3.86
"(20, 30]",24,3.43
"(30, 40]",2,0.29
"(40, 50]",1,0.14
"(50, 100]",2,0.29
"(100, 250]",0,0.00


  
column:  0 
buckets:  [0, 5, 10, 15, 20, 30, 40, 50, 100, 250, 500]


C:\Users\JGarza\AppData\Local\Temp\ipykernel_30736\3656564501.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  idf = idf.groupby(by='bucket').count()


,0,percent
bucket,,
"(0, 5]",0,0.00
"(5, 10]",26,3.71
"(10, 15]",473,67.57
"(15, 20]",135,19.29
"(20, 30]",40,5.71
"(30, 40]",8,1.14
"(40, 50]",5,0.71
"(50, 100]",7,1.00
"(100, 250]",5,0.71
